In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#input: data, expected peaks (substrate and metabolite)(dictionary?/two lists?
data = pd.read_csv('../Data/FA_20231113_2H_yeast_Pyruvate-d3_5.csv')

#expected peaks for different substrates
Pyruvate_compound = ['Substrate', 'metabolite1', 'metabolite2']
peaks_pyruvate= [2.468, 1.2261, 1.9775]

spectra_data= data.iloc[:,1:]
chem_shifts = data.iloc[:,0] 

---
---
# functions so far

In [3]:

def peak_fit(intensity, chem_shifts, threshold_percentile):
    """calculates possible peaks as minima of the second derivative and returns list of peaks

    Args:
        spectra_data (_type_): x data of spectra
        chem_shifts (_type_): y data of spectra
        threshold_percentile (int | float): Threshold above which the relevant data lies

    Returns:
        list: peak positions
    """
    threshold = np.percentile(intensity, threshold_percentile)
    first_derivative = np.gradient(intensity, chem_shifts)
    second_derivative = np.gradient(first_derivative, chem_shifts)
    third_derivative = np.gradient(second_derivative, chem_shifts)
    fourth_derivative = np.gradient(third_derivative, chem_shifts)

    sign_change = np.diff(np.sign(third_derivative)) != 0
    peak_mask = (intensity > threshold) & (second_derivative < 0) & (fourth_derivative > 0)
    peak_mask[1:] &= sign_change

    return chem_shifts[peak_mask].tolist()


In [4]:
def peakfit_sum(spectra_data, chem_shifts, threshold):
    """sums up all spectra and returns list of found peaks

    Args:
        filename (str): file with spectra data
        threshold_percentile (int | float): Threshold above which the relevant data lies
    """
    sum_of_spectra = np.sum(spectra_data, axis=1)
    # peak_pos = peak_fit(sum_of_spectra, chem_shifts, threshold)

    threshold = np.percentile(sum_of_spectra, threshold)

    first_derivative = np.gradient(sum_of_spectra, chem_shifts)
    second_derivative = np.gradient(first_derivative, chem_shifts)
    third_derivative = np.gradient(second_derivative, chem_shifts)
    fourth_derivative = np.gradient(third_derivative, chem_shifts)

    sign_change = np.diff(np.sign(third_derivative)) != 0
    peak_mask = (sum_of_spectra > threshold) & (second_derivative < 0) & (fourth_derivative > 0)
    peak_mask[1:] &= sign_change

    peak_pos= chem_shifts[peak_mask].tolist()

    return peak_pos

In [5]:
def normalize_water_singlet(data: pd.DataFrame):
    """
    Diese Funktion wird so angepasst, dass sie jedes Spektrum anhand eines bestimmten Peaks 'normalisiert'.
    Jedes Spektrum wird einzeln behandelt.
    """
    # Kopieren des DataFrames, um die ursprünglichen Daten nicht zu verändern
    data_normalized = pd.DataFrame()
    data_normalized['chem_shift'] = data.iloc[:,0]  # Chemische Verschiebungen beibehalten
    
    # Über jede Spektrumsspalte iterieren (jedes Spektrum)
    for col in data.columns[1:]:
        intensity = data[col]
        chem_shifts = data.iloc[:, 0]
        
        # Nutzung der peak_fit Funktion, um Peaks für jedes Spektrum zu ermitteln
        peak_pos = peak_fit(intensity.values, chem_shifts.values, 85)  # Annahme: threshold_percentile=85
        
        # Bestimmung des dem Wasserpeak am nächsten liegenden Peaks
        water = 4.7
        
        if peak_pos:
            closest_peak = min(peak_pos, key=lambda x: abs(x - water))
            
            # Adjustierung der chemischen Verschiebung basierend auf dem nächsten Peak
            adjusted_shifts = chem_shifts + (water - closest_peak)
            data_normalized[col] = intensity.set_axis(adjusted_shifts.index)
        else:
            # Falls keine Peaks gefunden wurden, Kopie der Originalintensitäten
            data_normalized[col] = intensity
    
    return data_normalized

In [6]:
def normalize_water_singlet(data: pd.DataFrame, threshold=85, delta_threshold=2, min_threshold=50):
    """
    Normalisiert jedes Spektrum im DataFrame basierend auf dem nächsten Wasserpeak, wobei der Threshold angepasst wird.
    
    Args:
        data (pd.DataFrame): Spectra-Daten.
        threshold (int, optional): Anfangsschwelle für die Peak-Suche. Standardmäßig 85.
        delta_threshold (int, optional): Reduktionsbetrag für den Threshold. Standardmäßig 2.
        min_threshold (int, optional): Minimale Schwelle für die Peak-Suche. Standardmäßig 50.
        
    Returns:
        pd.DataFrame: DataFrame mit normalisierten Daten.
    """
    data_normalized = pd.DataFrame()
    data_normalized['chem_shift'] = data.iloc[:,0]  
    
    # Iteration durch jede Spalte (jedes Spektrum).
    for col in data.columns[1:]:
        intensity = data[col]
        chem_shifts = data.iloc[:, 0]
        
        current_threshold = threshold
        peak_found = False
        while not peak_found and current_threshold >= min_threshold:
            # Nutzung der peak_fit Funktion mit dem aktuellen Threshold
            peak_pos = peak_fit(intensity.values, chem_shifts.values, current_threshold)
            
            # Suche nach Peaks im Wasserbereich
            water_peaks = [peak for peak in peak_pos if 4.6 <= peak <= 4.8]
            
            if water_peaks:
                closest_peak = min(water_peaks, key=lambda x: abs(x - 4.7))
                peak_found = True
            else:
                current_threshold -= delta_threshold
        
        # Wenn ein passender Peak gefunden wurde, chemische Schichten adjustieren
        if peak_found:
            adjusted_shifts = chem_shifts + (4.7 - closest_peak)
            data_normalized[col] = intensity.set_axis(adjusted_shifts.index)
        else:
            # Sicherheitsfallback, falls kein Peak gefunden wurde
            print(f"Kein Peak für {col} im Bereich von 4.6 bis 4.8 gefunden, auch nicht mit min_threshold={min_threshold}.")
            data_normalized[col] = intensity  # Originalintensitäten übernehmen
    
    return data_normalized

In [7]:
def normalize_water(data:pd.DataFrame):
    """adjusts the data to the chem_shift(water) = 4.7, based on the closest peak in summed up spectra
    --> needs existing function 'peakfit_sum', peak_fit

    Args:
        data (pd.DataFrame): spectra data with chem_shift in first column

    Returns:
        data_normalized (pd.DataFrame): dataframe with normalized data
    """

    spectra_data= data.iloc[:,1:]
    chem_shifts = data.iloc[:,0] 

    peak_pos = peakfit_sum(spectra_data, chem_shifts, 85)

    #identify water peak (closest to 4.7
    water = 4.7
    closest_peak = min(peak_pos, key=lambda x: abs(x - water))

    # reposition the whole spectra
    data_normalized = pd.DataFrame(chem_shifts.copy() + ( 4.7 - closest_peak))
    data_normalized = pd.concat([data_normalized, data.iloc[:, 1:]], axis=1)

    # Neubenennung der Spalten, um die Originalstruktur zu erhalten
    data_normalized.columns = data.columns

    return data_normalized

In [8]:
def peak_identify(data_normalized: pd.DataFrame, expected_peaks: list,  compound_names: list, initial_threshold = 85, max_shift = 0.15):
    """searches for peaks and matches them to the closest expected peak respecting the maximal shift. 
    Unidentified peaks are added to other(list). 
    Threshold is lowered until all expected peaks are found or number of other peaks succeeds numer of expected peaks
    
    needs existing function 'peakfit_sum'
    
    Args:
        data_normalized (pd.DataFrame): _description_
        compound_names (list): list with names of expected compunds (optional)
        expected_peaks (list): list with positions of expected peaks
        max_shift: maximum distance to expected peak position for 

    Returns:
        found (list): list with identified peaks most likely the expected ones
        other (list):  list with other found peaks
    """

    spectra_data= data_normalized.iloc[:,1:]
    chem_shifts = data_normalized.iloc[:,0]

    #create enpty lists for found and identified peaks
    found = [None] * len(expected_peaks)
    other = []
    threshold = initial_threshold

    while None in found or len(other) <= len(found):
        
        #search for peaks above given threshold
        detected_peaks = peakfit_sum(spectra_data, chem_shifts, threshold)
        
        for peak in detected_peaks:
            distances = [abs(peak - expected_peak) for expected_peak in expected_peaks]
            min_distance = min(distances)
            index = distances.index(min_distance)
            
            if min_distance <= max_shift:
                if found[index] is None:
                    found[index] = peak
                elif found[index] != peak:
                    other.append(peak)
            elif min_distance > max_shift:
                other.append(peak)
            
        threshold -= 2

        if threshold < 0 or None not in found or len(other) > len(found):
            if None not in found:
                #print('alle peaks gefunden')
                break
            else: 
                break

    print("Found Peaks: ", found)
    print("Other Peaks: ", other)

    return found, other

In [9]:
def peak_df(data, expected_peaks, compound_names, shift_tol=0.01, viz=True, min_cols_per_section = 20):

    # normalize data
    df = normalize_water(data)
   
    # Split into section with at least 20 columns each
    cols_to_divide = len(df.columns)-1 #-1 because first column is chemical shift
    num_sections = max(cols_to_divide // min_cols_per_section, 1) if cols_to_divide > min_cols_per_section else 1

    cols_per_section = cols_to_divide // num_sections
    extra_cols = cols_to_divide % num_sections  # Extra columns to distribute

    sections = []
    start_col = 1 #ignore first column
    col_sect = []

    for section in range(num_sections):
        extra = 1 if section < extra_cols else 0
        end_col = start_col + cols_per_section + extra
        #sections.append(df.iloc[:, start_col:end_col])
        sections.append(df.iloc[:, [0] + list(range(start_col, end_col))]) #add column with chem_shift to each section
        start_col = end_col
        col_sect.extend([f"{section + 1}"] * (cols_per_section + extra)) #column entries for final dataframe

    found_lists = []
    other_lists = []

    # Find peaks for each section
    for df_section in sections:
        found, other = peak_identify(df_section, expected_peaks, compound_names)
        found_lists.append(found)
        other_lists.append(other)

    
    #create final dataframe
    final_df = pd.DataFrame({
        'Column': df.columns[1:],  # Exclude the chemical shift from the final DataFrame columns
        'section': col_sect
    })

    
    peaks_data = {'Found Peaks': [], 'Other Peaks': []}
    for i in range(num_sections):
        section_length = len(sections[i].columns) - 1  # -1 to ignore the chemical shift column
        peaks_data['Found Peaks'].extend([found_lists[i]] * section_length)
        peaks_data['Other Peaks'].extend([other_lists[i]] * section_length)

    final_df['Found Peaks'] = peaks_data['Found Peaks']
    final_df['Other Peaks'] = peaks_data['Other Peaks']

    return final_df

In [10]:
peak_df = peak_df(data, expected_peaks=peaks_pyruvate, compound_names= Pyruvate_compound)

Found Peaks:  [2.4275300000000004, 1.2261300000000004, 1.9673300000000005]
Other Peaks:  [1.5356500000000004, 2.2646300000000004, 4.68778]
Found Peaks:  [2.4275300000000004, 1.2261300000000004, 1.9673300000000005]
Other Peaks:  [1.3849600000000004, 1.5356500000000004, 4.68778]
Found Peaks:  [2.4275300000000004, 1.2261300000000004, 1.9714100000000003]
Other Peaks:  [1.3890400000000005, 1.5356500000000004, 4.6918500000000005]
Found Peaks:  [2.4275300000000004, 1.2302100000000005, 1.9714100000000003]
Other Peaks:  [1.0673100000000004, 1.3849600000000004, 1.5356500000000004, 4.70814]
Found Peaks:  [2.4275300000000004, 1.2302100000000005, 1.9714100000000003]
Other Peaks:  [1.0713800000000004, 1.3808900000000004, 1.5356500000000004, 4.70814]
Found Peaks:  [2.4275300000000004, 1.2302100000000005, 1.9754800000000003]
Other Peaks:  [1.0713800000000004, 1.3849600000000004, 1.5356500000000004, 4.716290000000001]


In [25]:
print(peak_df['Other Peaks'].loc[127])#[127])

[1.0713800000000004, 1.3849600000000004, 1.5356500000000004, 4.716290000000001]


---
---
# as a class

In [11]:
class SpectraAnalysis:
    def __init__(self):
        pass

    def peakfit_sum(self, spectra_data, chem_shifts, threshold):
        """Summen der Spektren und Rückgabe einer Liste gefundener Peaks"""
        sum_of_spectra = np.sum(spectra_data, axis=1)
        threshold = np.percentile(sum_of_spectra, threshold)

        first_derivative = np.gradient(sum_of_spectra, chem_shifts)
        second_derivative = np.gradient(first_derivative, chem_shifts)
        third_derivative = np.gradient(second_derivative, chem_shifts)
        fourth_derivative = np.gradient(third_derivative, chem_shifts)

        sign_change = np.diff(np.sign(third_derivative)) != 0
        peak_mask = (sum_of_spectra > threshold) & (second_derivative < 0) & (fourth_derivative > 0)
        peak_mask[1:] &= sign_change

        peak_pos = chem_shifts[peak_mask].tolist()
        return peak_pos

    def normalize_water(self, data):
        """Anpassung der Daten basierend auf dem Wasser-Peak"""
        spectra_data = data.iloc[:, 1:]
        chem_shifts = data.iloc[:, 0]

        peak_pos = self.peakfit_sum(spectra_data, chem_shifts, 85)
        water = 4.7
        closest_peak = min(peak_pos, key=lambda x: abs(x - water))

        data_normalized = pd.DataFrame(chem_shifts.copy() + (4.7 - closest_peak))
        data_normalized = pd.concat([data_normalized, data.iloc[:, 1:]], axis=1)
        data_normalized.columns = data.columns
        return data_normalized

    def peak_identify(self, data_normalized, expected_peaks, compound_names, initial_threshold=85, max_shift=0.15):
        """Identifizierung von Peaks und Zuordnung zu erwarteten Peaks"""
        spectra_data = data_normalized.iloc[:, 1:]
        chem_shifts = data_normalized.iloc[:, 0]

        found = [None] * len(expected_peaks)
        other = []
        threshold = initial_threshold

        while None in found or len(other) <= len(found):
            detected_peaks = self.peakfit_sum(spectra_data, chem_shifts, threshold)
            for peak in detected_peaks:
                distances = [abs(peak - expected_peak) for expected_peak in expected_peaks]
                min_distance = min(distances)
                index = distances.index(min_distance)

                if min_distance <= max_shift:
                    if found[index] is None:
                        found[index] = peak
                    elif found[index] != peak:
                        other.append(peak)
                else:
                    other.append(peak)

            threshold -= 2
            if threshold < 0 or None not in found or len(other) > len(found):
                break

        print("Found Peaks: ", found)
        print("Other Peaks: ", other)

        return found, other

    def peak_df(self, data, expected_peaks, compound_names, min_cols_per_section=20):
        """creates dataframe with expected peaks and other found peaks"""
        #normalize data
        df = self.normalize_water(data)
    
        # Split into section with at least 20 columns each
        cols_to_divide = len(df.columns)-1 #-1 because first column is chemical shift
        num_sections = max(cols_to_divide // min_cols_per_section, 1) if cols_to_divide > min_cols_per_section else 1

        cols_per_section = cols_to_divide // num_sections
        extra_cols = cols_to_divide % num_sections  # Extra columns to distribute

        sections = []
        start_col = 1 #ignore first column
        col_sect = []

        for section in range(num_sections):
            extra = 1 if section < extra_cols else 0
            end_col = start_col + cols_per_section + extra
            #sections.append(df.iloc[:, start_col:end_col])
            sections.append(df.iloc[:, [0] + list(range(start_col, end_col))]) #add column with chem_shift to each section
            start_col = end_col
            col_sect.extend([f"{section + 1}"] * (cols_per_section + extra)) #column entries for final dataframe

        found_lists = []
        other_lists = []

        # Find peaks for each section
        for df_section in sections:
            found, other = self.peak_identify(df_section, expected_peaks, compound_names)
            found_lists.append(found)
            other_lists.append(other)

        
        #create final dataframe
        final_df = pd.DataFrame({
            'Column': df.columns[1:],  # Exclude the chemical shift from the final DataFrame columns
            'section': col_sect
        })

        
        peaks_data = {'Found Peaks': [], 'Other Peaks': []}
        for i in range(num_sections):
            section_length = len(sections[i].columns) - 1  # -1 to ignore the chemical shift column
            peaks_data['Found Peaks'].extend([found_lists[i]] * section_length)
            peaks_data['Other Peaks'].extend([other_lists[i]] * section_length)

        final_df['Found Peaks'] = peaks_data['Found Peaks']
        final_df['Other Peaks'] = peaks_data['Other Peaks']

        return final_df


In [12]:
peak_positions = analyser.peakfit_sum(spectra_data, chem_shifts, threshold)

NameError: name 'analyser' is not defined

In [ ]:
peak_df(data, expected_peaks=peaks_pyruvate, compound_names= Pyruvate_compound)

In [ ]:
#test
import sys
sys.path.append('../app')

from peakpos_df import SpectraAnalysis
# Jetzt können Sie SpectraAnalysis nutzen
analyser = SpectraAnalysis()
peak_positions = analyser.peak_df(data, expected_peaks=peaks_pyruvate, compound_names= Pyruvate_compound)

Found Peaks:  [2.4275300000000004, 1.2261300000000004, 1.9673300000000005]
Other Peaks:  [1.5356500000000004, 2.2646300000000004, 4.68778]
Found Peaks:  [2.4275300000000004, 1.2261300000000004, 1.9673300000000005]
Other Peaks:  [1.3849600000000004, 1.5356500000000004, 4.68778]
Found Peaks:  [2.4275300000000004, 1.2261300000000004, 1.9714100000000003]
Other Peaks:  [1.3890400000000005, 1.5356500000000004, 4.6918500000000005]
Found Peaks:  [2.4275300000000004, 1.2302100000000005, 1.9714100000000003]
Other Peaks:  [1.0673100000000004, 1.3849600000000004, 1.5356500000000004, 4.70814]
Found Peaks:  [2.4275300000000004, 1.2302100000000005, 1.9714100000000003]
Other Peaks:  [1.0713800000000004, 1.3808900000000004, 1.5356500000000004, 4.70814]
Found Peaks:  [2.4275300000000004, 1.2302100000000005, 1.9754800000000003]
Other Peaks:  [1.0713800000000004, 1.3849600000000004, 1.5356500000000004, 4.716290000000001]


In [ ]:
display(peak_positions)

,Column,section,Found Peaks,Other Peaks
0,FA_20231113_2H_yeast_1.5.ser#1,1,"[2.4275300000000004, 1.2261300000000004, 1.967...","[1.5356500000000004, 2.2646300000000004, 4.68778]"
1,FA_20231113_2H_yeast_1.5.ser#2,1,"[2.4275300000000004, 1.2261300000000004, 1.967...","[1.5356500000000004, 2.2646300000000004, 4.68778]"
2,FA_20231113_2H_yeast_1.5.ser#3,1,"[2.4275300000000004, 1.2261300000000004, 1.967...","[1.5356500000000004, 2.2646300000000004, 4.68778]"
3,FA_20231113_2H_yeast_1.5.ser#4,1,"[2.4275300000000004, 1.2261300000000004, 1.967...","[1.5356500000000004, 2.2646300000000004, 4.68778]"
4,FA_20231113_2H_yeast_1.5.ser#5,1,"[2.4275300000000004, 1.2261300000000004, 1.967...","[1.5356500000000004, 2.2646300000000004, 4.68778]"
...,...,...,...,...
125,FA_20231113_2H_yeast_1.5.ser#126,6,"[2.4275300000000004, 1.2302100000000005, 1.975...","[1.0713800000000004, 1.3849600000000004, 1.535..."
126,FA_20231113_2H_yeast_1.5.ser#127,6,"[2.4275300000000004, 1.2302100000000005, 1.975...","[1.0713800000000004, 1.3849600000000004, 1.535..."
127,FA_20231113_2H_yeast_1.5.ser#128,6,"[2.4275300000000004, 1.2302100000000005, 1.975...","[1.0713800000000004, 1.3849600000000004, 1.535..."
128,FA_20231113_2H_yeast_1.5.ser#129,6,"[2.4275300000000004, 1.2302100000000005, 1.975...","[1.0713800000000004, 1.3849600000000004, 1.535..."
